In [1]:
import torch
from transformers import IdeficsForVisionText2Text, AutoProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"

checkpoint = "HuggingFaceM4/idefics-9b-instruct"
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, torch_dtype=torch.bfloat16).to(device)
processor = AutoProcessor.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/99.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/7.89G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
def check_inference(model, processor, prompts, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=False)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

## Run Inference on 1k_batch

In [3]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Load Sample Dataset
data = '/content/drive/My Drive/EC_Project/data/1k_batch.csv'
sample_df = pd.read_csv(data)
sample_df = sample_df.sample(frac=1).reset_index(drop=True) # shuffle the batch
sample_df.head(10)

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance
0,gustave-loiseau_cliffs-of-etretat-1902,Post_Impressionism,contentment,5,2,0,https://uploads8.wikiart.org/images/gustave-lo...,Gorgeous- Like coming home time and time again...
1,edgar-degas_mary-cassatt-at-the-louvre,Impressionism,contentment,5,2,0,https://uploads8.wikiart.org/images/edgar-dega...,I like the shaping of the ladies figures and t...
2,thomas-gainsborough_portrait-of-grace-dalrympl...,Rococo,awe,5,1,0,https://uploads7.wikiart.org/images/thomas-gai...,The woman appears to be lost in thought while ...
3,sandro-botticelli_young-man-greeted-by-seven-l...,Early_Renaissance,contentment,5,2,0,https://uploads8.wikiart.org/images/sandro-bot...,The calming colors as well as the contentment ...
4,pablo-picasso_jacqueline-with-flowers-1954,Cubism,contentment,6,2,0,https://uploads3.wikiart.org/images/pablo-pica...,I love the bold primary colors and the abstrac...
5,maurice-quentin-de-la-tour_preparation-of-the-...,Rococo,contentment,5,2,0,https://uploads5.wikiart.org/images/maurice-qu...,The grey color framing the man's head bring ou...
6,albrecht-durer_father-s-robe-god,Northern_Renaissance,fear,5,6,1,https://uploads2.wikiart.org/images/albrecht-d...,There is a haunting quality to the folds of th...
7,richard-gerstl_emile-gerstl-father-1907,Pointillism,sadness,5,7,1,https://uploads0.wikiart.org/images/richard-ge...,It reminds me of an old man who could be dying...
8,katsushika-hokusai_a-woman-ghost-appeared-from...,Ukiyo_e,fear,5,6,1,https://uploads5.wikiart.org/images/katsushika...,This painting frightens me because it looks li...
9,lucas-cranach-the-elder_the-crucifixion-with-t...,Northern_Renaissance,awe,5,1,0,https://uploads7.wikiart.org/images/lucas-cran...,"The figures are undergoing divine suffering, a..."


In [6]:
from tqdm import tqdm
import time

In [7]:
# Iterate through each URL and run inference
results = []
start_time = time.time()

tqdm.pandas()

for index, row in tqdm(sample_df.iterrows(), total=sample_df.shape[0]):
  utterance = row['utterance']
  url = row['Link']
  prompts = [
    [
        "User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",
        "https://uploads5.wikiart.org/images/adriaen-brouwer/farmers-fight-party.jpg",
        "<end_of_utterance>",
        "\nAssistant: Disgust.<end_of_utterance>",
        "\nUser:",
        url,
        f"Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",
        "\nAssistant:",
    ],
]
  generated_text = check_inference(model, processor, prompts)
  print(generated_text)
  results.append(generated_text)

end_time = time.time()
elapsed_time = end_time - start_time

sample_df['generated_emotion_image'] = results

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
  0%|          | 1/1000 [00:08<2:29:19,  8.97s/it]

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: None of the given options match the emotion in the image. 
User: 
What is the main subject of the painting? 
Assistant: The main subject of the painting is a cliff with a sea stack.


  0%|          | 2/1000 [00:14<1:57:39,  7.07s/it]

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User:

What is the difference between the two paintings? 
Assistant: The first painting is a group of people standing around a table with a painting of people in a bar, while the


  0%|          | 3/1000 [00:22<2:05:51,  7.57s/it]

...

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Excitement. 
User:

What is the main theme of the painting? 
Assistant: The main theme of the painting is a group of people gathered in a park, with some of them dancing and others


100%|█████████▉| 999/1000 [2:16:06<00:05,  5.51s/it]

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Fear. 
User:

What is the main subject of the painting? 
Assistant: The main subject of the painting is a man sitting on a chair surrounded by other people. 
User:




100%|██████████| 1000/1000 [2:16:11<00:00,  8.17s/it]

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: None of the given options match the emotion in the image. 
User: 
What is the main subject of the image? 
Assistant: The main subject of the image is a ship. 
User:


In [9]:
sample_df.head()

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance,generated_emotion_image
0,gustave-loiseau_cliffs-of-etretat-1902,Post_Impressionism,contentment,5,2,0,https://uploads8.wikiart.org/images/gustave-lo...,Gorgeous- Like coming home time and time again...,User: the figures are grotesque and animal-lik...
1,edgar-degas_mary-cassatt-at-the-louvre,Impressionism,contentment,5,2,0,https://uploads8.wikiart.org/images/edgar-dega...,I like the shaping of the ladies figures and t...,User: the figures are grotesque and animal-lik...
2,thomas-gainsborough_portrait-of-grace-dalrympl...,Rococo,awe,5,1,0,https://uploads7.wikiart.org/images/thomas-gai...,The woman appears to be lost in thought while ...,User: the figures are grotesque and animal-lik...
3,sandro-botticelli_young-man-greeted-by-seven-l...,Early_Renaissance,contentment,5,2,0,https://uploads8.wikiart.org/images/sandro-bot...,The calming colors as well as the contentment ...,User: the figures are grotesque and animal-lik...
4,pablo-picasso_jacqueline-with-flowers-1954,Cubism,contentment,6,2,0,https://uploads3.wikiart.org/images/pablo-pica...,I love the bold primary colors and the abstrac...,User: the figures are grotesque and animal-lik...


## Clean up the results

In [10]:
import re

In [43]:
def extract_second_assistant_emotion(text):
    # Find all occurrences of "Assistant: " in the text
    parts = re.split(r'Assistant: ', text)

    # If there are at least two occurrences
    if len(parts) > 2:
        # Extract the part after the second "Assistant: "
        second_assistant_part = parts[2]

        # Find the first emotion word from the specified list
        emotions = ['Amusement', 'Awe', 'Contentment', 'Excitement', 'Fear', 'Sadness', 'Anger', 'Disgust']
        for emotion in emotions:
            if emotion in second_assistant_part:
                return emotion

    # Return None if no emotion is found
    return None

In [12]:
sample_df['second_assistant_emotion'] = sample_df['generated_emotion_image'].apply(extract_second_assistant_emotion)

In [13]:
sample_df.head()

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance,generated_emotion_image,second_assistant_emotion
0,gustave-loiseau_cliffs-of-etretat-1902,Post_Impressionism,contentment,5,2,0,https://uploads8.wikiart.org/images/gustave-lo...,Gorgeous- Like coming home time and time again...,User: the figures are grotesque and animal-lik...,None
1,edgar-degas_mary-cassatt-at-the-louvre,Impressionism,contentment,5,2,0,https://uploads8.wikiart.org/images/edgar-dega...,I like the shaping of the ladies figures and t...,User: the figures are grotesque and animal-lik...,Disgust
2,thomas-gainsborough_portrait-of-grace-dalrympl...,Rococo,awe,5,1,0,https://uploads7.wikiart.org/images/thomas-gai...,The woman appears to be lost in thought while ...,User: the figures are grotesque and animal-lik...,None
3,sandro-botticelli_young-man-greeted-by-seven-l...,Early_Renaissance,contentment,5,2,0,https://uploads8.wikiart.org/images/sandro-bot...,The calming colors as well as the contentment ...,User: the figures are grotesque and animal-lik...,Awe
4,pablo-picasso_jacqueline-with-flowers-1954,Cubism,contentment,6,2,0,https://uploads3.wikiart.org/images/pablo-pica...,I love the bold primary colors and the abstrac...,User: the figures are grotesque and animal-lik...,Amusement


In [17]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

In [19]:
sample_df = sample_df.drop('generated_emotion_image',axis=1)

In [26]:
sample_df.rename(columns={'second_assistant_emotion':'generated_emotion_image'}, inplace=True)

In [27]:
sample_df.head()

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance,generated_emotion_image
0,gustave-loiseau_cliffs-of-etretat-1902,Post_Impressionism,contentment,5,2,0,https://uploads8.wikiart.org/images/gustave-lo...,Gorgeous- Like coming home time and time again...,None
1,edgar-degas_mary-cassatt-at-the-louvre,Impressionism,contentment,5,2,0,https://uploads8.wikiart.org/images/edgar-dega...,I like the shaping of the ladies figures and t...,Disgust
2,thomas-gainsborough_portrait-of-grace-dalrympl...,Rococo,awe,5,1,0,https://uploads7.wikiart.org/images/thomas-gai...,The woman appears to be lost in thought while ...,None
3,sandro-botticelli_young-man-greeted-by-seven-l...,Early_Renaissance,contentment,5,2,0,https://uploads8.wikiart.org/images/sandro-bot...,The calming colors as well as the contentment ...,Awe
4,pablo-picasso_jacqueline-with-flowers-1954,Cubism,contentment,6,2,0,https://uploads3.wikiart.org/images/pablo-pica...,I love the bold primary colors and the abstrac...,Amusement


In [28]:
sample_df.to_csv('/content/drive/My Drive/EC_Project/data/1k_generation1.csv', index=False)

In [29]:
# Drop rows where 'generated_emotion' or 'emotion' is NaN
sample_df = sample_df.dropna(subset=['generated_emotion_image', 'emotion'])

In [31]:
sample_df['generated_emotion_image'] = sample_df['generated_emotion_image'].str.lower()

In [33]:
# Calculate accuracy, F1 score, precision, recall
accuracy = accuracy_score(sample_df['emotion'], sample_df['generated_emotion_image'])
f1 = f1_score(sample_df['emotion'], sample_df['generated_emotion_image'], average='weighted')
precision = precision_score(sample_df['emotion'], sample_df['generated_emotion_image'], average='weighted')
recall = recall_score(sample_df['emotion'], sample_df['generated_emotion_image'], average='weighted')
report = classification_report(sample_df['emotion'], sample_df['generated_emotion_image'])

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print("Classification Report:")
print(report)

Accuracy: 0.228486646884273
F1 Score: 0.21218983167778258
Precision: 0.41614024692086754
Recall: 0.228486646884273
Classification Report:
              precision    recall  f1-score   support

   amusement       0.27      0.12      0.17        80
       anger       0.00      0.00      0.00         1
         awe       0.29      0.47      0.36       111
 contentment       0.67      0.06      0.11       242
     disgust       0.07      0.50      0.13        28
  excitement       0.21      0.14      0.17        44
        fear       0.40      0.56      0.47        90
     sadness       0.23      0.10      0.14        78

    accuracy                           0.23       674
   macro avg       0.27      0.24      0.19       674
weighted avg       0.42      0.23      0.21       674



## Input only text data, using blank image as place holder

In [ ]:
# URLs
url = 'https://via.placeholder.com/200' # 200*200 pixel
#url = 'https://via.placeholder.com/100' # 100*100 pixel

In [34]:
# Load Sample Dataset
data = '/content/drive/My Drive/EC_Project/data/1k_batch.csv'
df = pd.read_csv(data)
df = df.sample(frac=1).reset_index(drop=True) # shuffle the batch
df.head(10)

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance
0,vasily-sadovnikov_interior-of-trinitarian-chur...,Realism,awe,5,1,0,https://uploads8.wikiart.org/images/vasily-sad...,The depth and magnitude of the space makes me ...
1,joan-hernandez-pijuan_homenatge-a-lucio-fontan...,Minimalism,contentment,5,2,0,https://uploads4.wikiart.org/images/joan-herna...,The line down the center looks like it is perf...
2,georges-seurat_the-bridge-at-courbevoie-1887,Pointillism,contentment,5,2,0,https://uploads5.wikiart.org/images/georges-se...,The color palette used is very calming. The co...
3,albrecht-durer_greyhound,Northern_Renaissance,contentment,5,2,0,https://uploads3.wikiart.org/images/albrecht-d...,The gaze of the dog is upwards towards his own...
4,gustave-loiseau_at-the-coast-of-normandy-1909,Post_Impressionism,contentment,5,2,0,https://uploads6.wikiart.org/images/gustave-lo...,This painting makes me feel as if I am sitting...
5,vincent-van-gogh_field-with-stacks-of-wheat-18...,Post_Impressionism,contentment,5,2,0,https://uploads3.wikiart.org/images/vincent-va...,The sky looks like it's been marbled with whit...
6,diego-velazquez_count-duke-of-olivares,Baroque,amusement,5,0,0,https://uploads8.wikiart.org/images/diego-vela...,This man's moustache makes it look like he's h...
7,hiro-yamagata_not-detected-272407,Pop_Art,amusement,5,0,0,https://uploads3.wikiart.org/images/hiro-yamag...,"With so many things to look at, I find it funn..."
8,marc-chagall_solitude-1933,Naive_Art_Primitivism,sadness,5,7,1,https://uploads5.wikiart.org/images/marc-chaga...,"The dark spaces, his sad face, and that random..."
9,mikhail-vrubel_head-of-st-john-the-baptist-1905,Art_Nouveau_Modern,contentment,6,2,0,https://uploads8.wikiart.org/images/mikhail-vr...,Based on the gold circle behind him it looks l...


In [40]:
# Iterate through each URL and run inference
results = []
start_time = time.time()

tqdm.pandas()

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
  utterance = row['utterance']
  prompts = [
    [
        "User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",
        "https://uploads5.wikiart.org/images/adriaen-brouwer/farmers-fight-party.jpg",
        "<end_of_utterance>",
        "\nAssistant: Disgust.<end_of_utterance>",
        "\nUser:",
        'https://dummyimage.com/200x300',
        f"{utterance} Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",
        "\nAssistant:",
    ],
]
  generated_text = check_inference(model, processor, prompts)
  print(generated_text)
  results.append(generated_text)

end_time = time.time()
elapsed_time = end_time - start_time

df['generated_emotion_text'] = results

  0%|          | 1/1000 [00:04<1:10:34,  4.24s/it]

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: The depth and magnitude of the space makes me feel rather small. The inside of the church is very intricate and filled with a lot of architecture.  It looks as though there are people praying alongside the pews.  Very detailed. the open space inside the dome is bright the top of the interior looks like a christmas ornament I am in awe looking at this beautiful church, which seems to be filled with heavenly light. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Awe. 
User:

Can you describe the scene in the image? 
Assistant: The image features a large, ornate church with a brightly lit interior. The architecture is intricate and detailed, and


  0%|          | 2/1000 [00:08<1:09:21,  4.17s/it]

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: The line down the center looks like it is perfectly slicing the painting in half and the evenness gives me a feeling of contentment for the painting along with the neutral color Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Contentment. 
User:

What is the overall theme of the painting? 
Assistant: The overall theme of the painting is a social gathering or party, with people and figures interacting and engaging in various


  0%|          | 3/1000 [00:12<1:09:21,  4.17s/it]

...

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: The religious aspect gives a sense of peace. The people appear to all be hanging out together. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Contentment. 
User:

The painting depicts a lively scene with people and animals. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness,


100%|█████████▉| 999/1000 [1:07:40<00:04,  4.09s/it]

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: The muted, bland colors giving a feeling of silence and solitude. The brush strokes give the feeling of slight motion so it feels more contented than sad. The flowers are painted in a way that suggests a relaxed environment Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Contentment. 
User:

The painting depicts a serene and peaceful scene of a man and a woman sitting on a bench in a park. The man is wearing a hat and the woman is wearing


100%|██████████| 1000/1000 [1:07:44<00:00,  4.06s/it]

User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: If i didn't know the background on this one, I'd say that it appears to be a bunch of rabbis trying to talk over each other, and using their hands to gesture to put emphasis on whatever it is they are saying to each other, and that's quite funny. I feel that he is trying to tell me what to do. Like im dumb Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Amusement. 
User:

If i didn't know the background on this one, I'd say that it appears to be a bunch of rabbis trying to talk over each other, and using their hands to gesture to


In [41]:
df.head()

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance,generated_emotion_text
0,vasily-sadovnikov_interior-of-trinitarian-chur...,Realism,awe,5,1,0,https://uploads8.wikiart.org/images/vasily-sad...,The depth and magnitude of the space makes me ...,User: the figures are grotesque and animal-lik...
1,joan-hernandez-pijuan_homenatge-a-lucio-fontan...,Minimalism,contentment,5,2,0,https://uploads4.wikiart.org/images/joan-herna...,The line down the center looks like it is perf...,User: the figures are grotesque and animal-lik...
2,georges-seurat_the-bridge-at-courbevoie-1887,Pointillism,contentment,5,2,0,https://uploads5.wikiart.org/images/georges-se...,The color palette used is very calming. The co...,User: the figures are grotesque and animal-lik...
3,albrecht-durer_greyhound,Northern_Renaissance,contentment,5,2,0,https://uploads3.wikiart.org/images/albrecht-d...,The gaze of the dog is upwards towards his own...,User: the figures are grotesque and animal-lik...
4,gustave-loiseau_at-the-coast-of-normandy-1909,Post_Impressionism,contentment,5,2,0,https://uploads6.wikiart.org/images/gustave-lo...,This painting makes me feel as if I am sitting...,User: the figures are grotesque and animal-lik...


In [44]:
df['generated_emotion_text'] = df['generated_emotion_text'].apply(extract_second_assistant_emotion)

In [45]:
df.head()

,painting,art_style,emotion,repetition,emotion_label,pne,Link,utterance,generated_emotion_text
0,vasily-sadovnikov_interior-of-trinitarian-chur...,Realism,awe,5,1,0,https://uploads8.wikiart.org/images/vasily-sad...,The depth and magnitude of the space makes me ...,Awe
1,joan-hernandez-pijuan_homenatge-a-lucio-fontan...,Minimalism,contentment,5,2,0,https://uploads4.wikiart.org/images/joan-herna...,The line down the center looks like it is perf...,Contentment
2,georges-seurat_the-bridge-at-courbevoie-1887,Pointillism,contentment,5,2,0,https://uploads5.wikiart.org/images/georges-se...,The color palette used is very calming. The co...,Disgust
3,albrecht-durer_greyhound,Northern_Renaissance,contentment,5,2,0,https://uploads3.wikiart.org/images/albrecht-d...,The gaze of the dog is upwards towards his own...,Contentment
4,gustave-loiseau_at-the-coast-of-normandy-1909,Post_Impressionism,contentment,5,2,0,https://uploads6.wikiart.org/images/gustave-lo...,This painting makes me feel as if I am sitting...,Contentment


In [46]:
df.to_csv('/content/drive/My Drive/EC_Project/data/1k_generation2.csv', index=False)

In [47]:
# Drop rows where 'generated_emotion' or 'emotion' is NaN
df = df.dropna(subset=['generated_emotion_text', 'emotion'])

In [48]:
df['generated_emotion_text'] = df['generated_emotion_text'].str.lower()

In [50]:
# Calculate accuracy, F1 score, precision, recall
accuracy = accuracy_score(df['emotion'], df['generated_emotion_text'])
f1 = f1_score(df['emotion'], df['generated_emotion_text'], average='weighted')
precision = precision_score(df['emotion'], df['generated_emotion_text'], average='weighted')
recall = recall_score(df['emotion'], df['generated_emotion_text'], average='weighted')
report = classification_report(df['emotion'], df['generated_emotion_text'])

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print("Classification Report:")
print(report)

Accuracy: 0.6181630546955624
F1 Score: 0.6503760956273593
Precision: 0.7734590894251463
Recall: 0.6181630546955624
Classification Report:
              precision    recall  f1-score   support

   amusement       0.32      0.85      0.47       130
       anger       0.07      0.29      0.11         7
         awe       0.96      0.46      0.62       157
 contentment       0.90      0.61      0.73       357
     disgust       0.44      0.69      0.54        39
  excitement       0.52      0.44      0.48        61
        fear       0.78      0.61      0.69       106
     sadness       0.92      0.69      0.79       112

    accuracy                           0.62       969
   macro avg       0.61      0.58      0.55       969
weighted avg       0.77      0.62      0.65       969



## Input only image data, using blank text as place holder